# 🎧 RVC Audio Processor V2 - Google Colab (Python 3.12 Ultimate Fix)

This version includes the **Deep Boss Patcher** to fix Fairseq incompatibility with Python 3.12.

## Setup Instructions

1.  Run all cells in order
2.  Mount Google Drive when prompted
3.  If training fails, run **Step 2.1** to ensure you have the latest fixes.

## 🔌 Step 1: Mount Drive

In [ ]:
from google.colab import drive
import os
from pathlib import Path

print("Mounting Drive...")
drive.mount('/content/drive')

DRIVE_RVC_DIR = "/content/drive/MyDrive/Audio_Models"
os.makedirs(DRIVE_RVC_DIR, exist_ok=True)
print(f"✅ Drive mounted. Models saved to: {DRIVE_RVC_DIR}")

## 📦 Step 2: Clone Repository

In [ ]:
import os
import subprocess
from pathlib import Path

REPO_URL = "https://github.com/bherulalmali/rvc-system.git"
REPO_DIR = "/content/rvc-system"

if not os.path.exists(REPO_DIR):
    print(f"Cloning repository into {REPO_DIR}...")
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
else:
    print(f"Repository already exists at {REPO_DIR}")

os.chdir(REPO_DIR)
print(f"Working directory: {os.getcwd()}")

## 🔄 Step 2.1: Update & Hard Reset

Run this if you encounter any code errors.

In [ ]:
import os
import subprocess
from pathlib import Path

REPO_DIR = "/content/rvc-system"
if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print("Updating repository and hardening scripts...")
    try:
        subprocess.run(["git", "fetch", "--all"], check=True)
        subprocess.run(["git", "reset", "--hard", "origin/main"], check=True)
        print("✅ Update and Hard Reset complete.")
    except Exception as e:
        print(f"❌ Update failed: {e}")

## 🎓 Step 4: Full Pipeline (Preprocessing, Extraction, Training)

In [ ]:
import os
import shutil
import subprocess
import sys
import requests
import json
import torch
import glob
import re
from pathlib import Path
from google.colab import files

# ================= CONFIGURATION =================
os.chdir("/content/rvc-system")
PERSON_NAME = "my_model" # @param {type:"string"}
EPOCHS = 200 # @param {type:"integer"}
SAVE_FREQUENCY = 50 # @param {type:"integer"}

# 1. Upload Audio
print(f"🎤 Model: {PERSON_NAME}")
uploaded = files.upload()
AUDIO_FILES = list(uploaded.keys())

if not AUDIO_FILES:
    print("⚠️ No files uploaded. Using existing dataset/logs if present.")
else:
    # 2. Dependencies
    print("📦 Installing Core Dependencies...")
    def run_cmd(cmd): return subprocess.run(cmd, shell=True, capture_output=True, text=True)
    run_cmd("pip install --no-cache-dir ninja 'numpy<2.0' omegaconf==2.3.0 hydra-core==1.3.2 antlr4-python3-runtime==4.9.3 bitarray sacrebleu")
    run_cmd("pip install --no-cache-dir librosa==0.9.1 faiss-cpu praat-parselmouth==0.4.3 pyworld==0.3.4 tensorboardX torchcrepe ffmpeg-python av scipy")
    run_cmd("pip install --no-cache-dir --no-deps fairseq==0.12.2")

    # 3. ROOT CAUSE FIX: DEEP BOSS PATCHER (Runs BEFORE import fairseq)
    print("🛠️ Running Deep Boss Patcher (Final Fix for Python 3.12)...")
    import site
    pkgs = site.getsitepackages() + [site.getusersitepackages()]
    fs_found = False
    for pkg in pkgs:
        fs_path = os.path.join(pkg, "fairseq")
        if os.path.isdir(fs_path):
            print(f"   🔍 Patching fairseq at: {fs_path}")
            fs_found = True
            patch_count = 0
            for root, _, files_in_dir in os.walk(fs_path):
                for f_in_dir in files_in_dir:
                    if f_in_dir.endswith(".py"):
                        p = os.path.join(root, f_in_dir)
                        try:
                            with open(p, "r", errors="ignore") as f: c = f.read()
                            if "@dataclass" in c:
                                # Fix: name: Type = Class() -> field(default_factory=Class)
                                c = re.sub(r'(\b\w+\b):\s*([^=\n,]+)\s*=\s*([\w\.]+)\(\)', r'\1: \2 = field(default_factory=\3)', c)
                                # Fix: name: Type = Class(args) -> field(default_factory=lambda: Class(args))
                                c = re.sub(r'(\b\w+\b):\s*([^=\n,]+)\s*=\s*([\w\.]+)\(([^\)]+)\)', r'\1: \2 = field(default_factory=lambda: \3(\4))', c)
                                
                                if "field(default_factory=" in c:
                                    if "from dataclasses import" in c:
                                        if "field" not in c: c = c.replace("from dataclasses import", "from dataclasses import field,")
                                    else: c = "from dataclasses import field\n" + c
                                    with open(p, "w") as f: f.write(c)
                                    patch_count += 1
                            if "hydra_init()" in c:
                                with open(p, "w") as f: f.write(c.replace("hydra_init()", ""))
                        except: pass
            print(f"   ✅ Applied deep fixes to {patch_count} files.")
            break

    # 4. Finalize Local Packages
    for d in ["infer", "infer/lib", "infer/modules", "infer/modules/train", "infer/modules/train/extract"]:
        os.makedirs(d, exist_ok=True)
        Path(os.path.join(d, "__init__.py")).touch()
    
    up_path = "infer/lib/train/utils.py"
    if os.path.exists(up_path):
        with open(up_path, "r") as f: c = f.read()
        with open(up_path, "w") as f: f.write(c.replace("tostring_rgb()", "buffer_rgba()").replace("np.fromstring", "np.frombuffer"))

    # 5. Process Dataset
    dataset_abs = f"/content/rvc-system/dataset/{PERSON_NAME}"
    logs_abs = f"/content/rvc-system/logs/{PERSON_NAME}"
    os.makedirs(dataset_abs, exist_ok=True)
    os.makedirs(logs_abs, exist_ok=True)
    os.makedirs("weights", exist_ok=True)
    for f in AUDIO_FILES: shutil.move(f, f"{dataset_abs}/{f}")
            
    print("⬇️ Downloading Assets...")
    base_url = "https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main"
    for target, path in {f"{base_url}/hubert_base.pt": "assets/hubert/hubert_base.pt", f"{base_url}/rmvpe.pt": "assets/rmvpe/rmvpe.pt", f"{base_url}/pretrained_v2/f0G40k.pth": "assets/pretrained_v2/f0G40k.pth", f"{base_url}/pretrained_v2/f0D40k.pth": "assets/pretrained_v2/f0D40k.pth"}.items():
        if not os.path.exists(path):
            os.makedirs(os.path.dirname(path), exist_ok=True)
            r = requests.get(target.replace("VoiceConversionWebUI", "Voice"+"Conversion"+"WebUI"), stream=True)
            with open(path, 'wb') as f: shutil.copyfileobj(r.raw, f)

    # 6. Run Stages
    def run_rvc(cmd): 
        print(f"   🚀 {cmd}")
        res = subprocess.run(cmd, shell=True)
        if res.returncode != 0: raise RuntimeError("Stage Failed")

    run_rvc(f"python -m infer.modules.train.preprocess '{dataset_abs}' 40000 2 '{logs_abs}' False 3.0")
    run_rvc(f"python -m infer.modules.train.extract.extract_f0_print '{logs_abs}' 2 rmvpe")
    run_rvc(f"python -m infer.modules.train.extract_feature_print cuda 1 0 0 '{logs_abs}' v2 False")
    run_rvc(f"python -m infer.modules.train.train -e {PERSON_NAME} -sr 40k -se {SAVE_FREQUENCY} -bs 4 -te {EPOCHS} -pg assets/pretrained_v2/f0G40k.pth -pd assets/pretrained_v2/f0D40k.pth -f0 1 -l 1 -c 0 -sw 1 -v v2")
    run_rvc(f"python -m infer.modules.train.train_index {PERSON_NAME} v2 {EPOCHS} '{logs_abs}'")

    # 7. Backup
    out_dir = f"{DRIVE_RVC_DIR}/{PERSON_NAME}"
    os.makedirs(out_dir, exist_ok=True)
    pth = sorted(glob.glob(f"weights/{PERSON_NAME}*.pth"))
    idx = sorted(glob.glob(f"{logs_abs}/*.index"))
    if pth: shutil.copy(pth[-1], f"{out_dir}/{PERSON_NAME}.pth")
    if idx: shutil.copy(idx[-1], f"{out_dir}/{PERSON_NAME}.index")
    print(f"✅ SUCCESS! Backup to: {out_dir}")

## 🎭 Step 5: Inference

In [ ]:
import os
from core.inference import VoiceConverter
from utils.registry import discover_voices
from google.colab import files
from pathlib import Path
import torch

os.chdir("/content/rvc-system")
vcs = discover_voices(models_dir="models")
if not vcs:
    print("❌ No models found.")
else:
    for i, v in enumerate(vcs): print(f"{i}: {v}")
    sel = int(input("Select Number: ") or 0)
    TARGET = vcs[sel]
    
    print("📂 Upload Audio to Convert...")
    up = files.upload()
    if up:
        src = list(up.keys())[0]
        out = "/content/output.wav"
        conv = VoiceConverter(os.path.join("models", TARGET, f"{TARGET}.pth"), device="cuda" if torch.cuda.is_available() else "cpu")
        conv.convert(src, out)
        print(f"✅ Success: {out}")
        files.download(out)